In [1]:
import pandas as pd
import numpy as np
import requests
import time
import re
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt

In [2]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
artists = pd.read_csv("artists.csv")
train.head()

,user,artist,plays
0,eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03,5a8e07d5-d932-4484-a7f7-e700793a9c94,554
1,44ce793a6cd9d20f13f4a576a818ef983314bb5d,a3a92047-be1c-4f3e-8960-c4f8570984df,81
2,da9cf3f557161d54b76f24db64be9cc76db008e3,eeb1195b-f213-4ce1-b28c-8565211f8e43,708
3,8fa49ab25d425edcf05d44bfc1d5aea895287d81,a1419808-65d3-4d40-998c-1a0bac65eabc,265
4,b85fcaef67d2669cd99b334b5e8c8705263db2cf,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432,220


In [34]:
grouped_by_user = train.groupby('user')
user_means = grouped_by_user['plays'].mean()
user_medians = grouped_by_user['plays'].median()
user_stds = grouped_by_user['plays'].std()
user_ids = user_means.index

user_stats = {}
for i in range(len(grouped_by_user)) :
    user_stats[user_ids[i]] = {
        'mean' : user_means[i],
        'std' : user_stds[i],
        'median' : user_medians[i]
    }

In [258]:
grouped_by_artist = train.groupby('artist')
artist_means = grouped_by_artist['plays'].mean()
artist_medians = grouped_by_artist['plays'].median()
artist_stds = grouped_by_artist['plays'].std()
artist_ids = artist_means.index

artist_stats = {}
for i in range(len(grouped_by_artist)) :
    artist_stats[artist_ids[i]] = {
        'mean' : artist_means[i],
        'std' : artist_stds[i],
        'median' : artist_medians[i]
    }

In [38]:
GLOBAL_MEAN = np.mean(train.plays)
GLOBAL_MEDIAN = np.median(train.plays)

In [415]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

def get_wiki_url(artist, search_engine) :
    if pd.isnull(artist) == False and is_ascii(artist) :
        fixArtist = artist.replace(" ","+").replace("&","%26")
        
        if search_engine == "Yahoo" :
            var = requests.get('https://search.yahoo.com/search;_ylt=A0oG7l7PeB5P3G0AKASl87UF?p=site:wikipedia.org+'+fixArtist+'+music+band+artist')
        elif search_engine == "Bing" :
            var = requests.get('http://www.bing.com/search?q=site%3Awikipedia.org+'+fixArtist+'+music+band+artist')
        elif search_engine == "AOL" :
            var = requests.get('http://search.aol.com/aol/search?enabled_terms=&s_it=comsearch&q=site%3Awikipedia.org+'+fixArtist+"+music+band+artist")
        else :
            sys.exit("Out of search engines!")
                               
        soup = BeautifulSoup(var.text, "html.parser")
        relevant = [link for link in [el.get("href") for el in soup.find_all('a')] if type(link) == unicode and re.search('^https://en.wikipedia.org/', link)]
        if len(relevant) > 0 :
            return relevant[0]
        else :
            return "Unknown"

    else :
        return "Unknown"

In [264]:
artist_list = list(set(artists.name))
#artist_count = 0

In [420]:
if artist_count < len(artist_list) :
    for j in range(artist_count,len(artist_list)) :
        print j

        artist_name = str(artist_list[j])

        if artist_name != "nan" :
            artist_id = artists[artists.name == artist_name]['artist'].values[0]

            #try :
            #    artist_stats[artist_id]['url'] = get_wiki_url(artist_name, "Yahoo")
            try : 
                artist_stats[artist_id]['url'] = get_wiki_url(artist_name, "Bing")
            #try : 
            #    artist_stats[artist_id]['url'] = get_wiki_url(artist_name, "AOL")
            except :
                break

        artist_count += 1
else :
    print "Already finished"

Already finished


In [427]:
missing_artists = []
for key in artist_stats.keys() :
    if len(artist_stats[key]) != 4 :
        missing_artists.append(key)

In [429]:
for missing_id in missing_artists :
    artist_name = artists[artists.artist == missing_id]['name'].values[0]
    try : 
        artist_stats[missing_id]['url'] = get_wiki_url(artist_name, "Bing")
    except :
        break

In [433]:
for key in artist_stats.keys() :
    artist_name = artists[artists.artist == key]['name'].values[0]
    artist_stats[key]['name'] = artist_name

In [448]:
import json
with open('artist_stats.txt', 'w') as outfile:
    json.dump(artist_stats, outfile)

In [444]:
keys = artist_stats.keys()
unique_te = np.unique(test.artist)
unique_tr = np.unique(train.artist)

for artist in unique_te :
    if artist not in keys:
        print artist
for artist in unique_tr :
    if artist not in keys:
        print artist

In [454]:
counter = 0

2000

In [456]:
if counter < len(artist_stats.keys()) :
    for key in artist_stats.keys()[counter:len(artist_stats.keys())] :
        url = artist_stats[key]['url']
        if url != "Unknown" :
            var = requests.get(url)
            soup = BeautifulSoup(var.text, "html.parser")

            genre_exists = soup.find(title='Music genre')
            if genre_exists :
                artist_stats[key]['genre'] = [entry.get("href") for entry in soup.find(title="Music genre").parent.parent.find_all('a')[1:] if entry.get("href")[1:5] == 'wiki']
            else :
                artist_stats[key]['genre'] = "Unknown"

            ya_exists = soup.find(text=re.compile("Years"))
            try :
                artist_stats[key]['ya'] = ya_exists.parent.parent.find('td').get_text()
            except AttributeError :
                artist_stats[key]['ya'] = "Unknown"
        else :
            artist_stats[key]['genre'] = 'Unknown'
            artist_stats[key]['ya'] = 'Unknown'
        print counter
        counter += 1
else :
    print "Already done!"

Already done!


In [462]:
import json
with open('artist_stats.txt', 'w') as outfile:
    json.dump(artist_stats, outfile)

In [8]:
import collections
blahblah = collections.defaultdict(int)
for artist in artists.name :
    blahblah[artist] += 1
    if blahblah[artist] > 1 and pd.isnull(artist) == False:
        print artist

Justice
Ben Harper
Camera Obscura
Nouvelle Vague
Enigma
Nirvana
Rhapsody of Fire
Moby
Feist
[Disney]
[unknown]
M.I.A.
Air


In [10]:
artists[artists.name == 'Ben Harper']

,artist,name
112,1582a5b8-538e-45e7-9ae4-4099439a0e79,Ben Harper
464,b07016ee-5de9-4419-8205-0490d95dc638,Ben Harper


In [13]:
import json
with open('artist_stats.txt') as data_file:    
    artist_stats = json.load(data_file)

In [17]:
artist_stats['1582a5b8-538e-45e7-9ae4-4099439a0e79']

{u'genre': [u'/wiki/Folk_rock',
  u'/wiki/Blues_rock',
  u'/wiki/Alternative_rock',
  u'/wiki/Soul_music',
  u'/wiki/Funk_rock',
  u'/wiki/Gospel_music',
  u'/wiki/Reggae',
  u'/wiki/Jam_rock',
  u'/wiki/Neofolk'],
 u'mean': 172.29055258467022,
 u'median': 91.0,
 u'name': u'Ben Harper',
 u'std': 255.0897238505667,
 u'url': u'https://en.wikipedia.org/wiki/Ben_Harper',
 u'ya': u'1992\u2013present'}